In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping

plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams.update({'font.size':20})
plt.rcParams["figure.figsize"] = (12,10)

In [2]:
str_files = ''
#Carrega os dados do arquivo .csv
ugrhi06 = pd.read_csv(str_files + "cetesb_concatenado06semoutliers.csv",encoding='utf-8',sep=',',index_col=[0])

#Carrega os dados do arquivo .csv
ugrhi10 = pd.read_csv(str_files + "cetesb_concatenado10semoutliers.csv",encoding='utf-8',sep=',',index_col=[0])
ugrhi10

,solido,ph,coliformes,dbo,fosforo,od,temperatura,turbidez
0,2.280559,0.908166,3.548615,0.772077,0.318589,0.540538,1.357776,1.456998
1,2.286276,0.908804,3.535570,0.761761,0.318589,0.536580,1.361072,1.461206
2,2.291917,0.908804,3.522122,0.751194,0.318589,0.532586,1.364343,1.465373
3,2.297487,0.909592,3.508243,0.740363,0.318589,0.528555,1.367589,1.469500
4,2.302986,0.909592,3.493907,0.729255,0.318589,0.524486,1.370811,1.473588
...,...,...,...,...,...,...,...,...
497,2.559008,0.910426,3.755951,0.929419,0.078276,0.681241,1.414973,1.477121
498,2.539782,0.910426,3.755951,0.983401,0.077413,0.685742,1.419129,1.540955
499,2.519664,0.914211,3.755951,1.031408,0.076549,0.690196,1.423246,1.596597
500,2.498569,0.910291,3.755951,1.074634,0.075684,0.694605,1.427324,1.645913


In [3]:
print('---- Tamanho dos conjuntos originais ----')
print('ugrhi06:',len(ugrhi06))
print('ugrhi10:',len(ugrhi10))

#Seleciona os dados equivalente ao tamanho da URGHI10
ugrhi06 = ugrhi06.iloc[0:len(ugrhi10)]

#Seleciona os valores da coluna ph da URGHI06, na qual será utilizado para a normalização dos dados quando apresentar o gráfico
#train_previsao = ugrhi06['ph'].values
#train_previsao = train_previsao.reshape(len(train_previsao),1)

print('---- Conjunto de dados y igualado ao tamanho de dados do conjunto X ----')
print('ugrhi06:',len(ugrhi06))
print('ugrhi10:',len(ugrhi10))

---- Tamanho dos conjuntos originais ----
ugrhi06: 503
ugrhi10: 502
---- Conjunto de dados y igualado ao tamanho de dados do conjunto X ----
ugrhi06: 502
ugrhi10: 502


In [4]:
def pega_index_coluna_ugrhi10(p):
    
    index_coluna = 0
    
    if (p == 'solido'):
        index_coluna = 0
    elif (p == 'ph'):
        index_coluna = 1
    elif (p == 'coliformes'):
        index_coluna = 2 
    elif (p == 'dbo'):
        index_coluna = 3
    elif (p == 'fosforo'):
        index_coluna = 4
    elif (p == 'od'):
        index_coluna = 5 
    elif (p == 'temperatura'):
        index_coluna = 6 
    elif (p == 'turbidez'):
        index_coluna = 7
        
    return index_coluna

def pegar_dados_coluna_predita_train_test(trainingd,percent,index_coluna):
    data = trainingd.iloc[:,:].values
    train = trainingd.iloc[0:int(len(data)*percent),:].values  
    train_previsao = trainingd.iloc[0:int(len(data)*percent),index_coluna:index_coluna+1].values
    test = trainingd.iloc[len(train):,index_coluna:index_coluna+1].values
    
    return train,train_previsao, test

def pegar_dados_coluna_predita_train_test_od(trainingd,percent,index_coluna):
    data = trainingd.iloc[:,:].values
    train = trainingd.iloc[0:int(len(data)*percent),:].values  
    train_previsao = trainingd.iloc[0:int(len(data)*percent),index_coluna:index_coluna+1].values
    test = trainingd.iloc[len(train):,index_coluna:index_coluna+1].values
    
    return train,train_previsao, test

#Normalização dos dados: Normaliza os dados dentro um intervalo (0 a 1).
def normalizacao(train,test):
    sc = MinMaxScaler()
    testd = test
    train = sc.fit_transform(train)
    test = sc.fit_transform(test)    
    return train,test,testd

#Prepara o conjunto de dados em X e y, considerando a janela de visualização (lags).
#cy = coluna que será predita
def prepara_dados(dados,lags,cy):
    X = []
    y = []
    for i in range(lags, len(dados)):
        X.append(dados[i-lags:i,:])
        y.append(dados[i, cy])
       
    return np.array(X), np.array(y)

#Calcula o MAPE
#Define função para calcular o MAPE
#def mape(y_pred,y_true):
 #   return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mape(y_pred,y_true):
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    '''  
    mape_sum = 0
    for real,prediction in zip(y_true,y_pred):
        mape_sum += (abs((real - prediction))/real)
        
        print(real)
        mape = mape_sum/len(real)
    '''
    return mape

def rmse(y_pred,y_true):
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    return rmse

def r2(y_pred,y_true):
    rscore = r2_score(y_true,y_pred)
    return rscore

def rquadrado(y_pred,y_true):
    #Soma Total dos Quadrados (STQ): mostra a variação de y em torno da própria média. 
    #É o somatório das diferenças entre o valor alvo real e sua média elevado ao quadrado.
    y_traco = np.mean(y_true)
    
    print('media y_true:', y_traco)
        
    stq = 0
    
    for s in y_true:
        a = s - y_traco
        st = a * a
        stq = stq + st
    
    print('Soma Total dos Quadrados (STQ):', stq[0])
    
    #Soma dos Quadrados dos Resíduos (SQU): variação de Y que não é explicada pelo modelo elaborado. 
    #É o somatório das diferenças entre o valor predito e o valor real elevados ao quadrado.
    squ = 0
    
    y_traco = np.mean(y_pred)
    print('media y_pred:', y_traco)
    
    for n in range(len(y_true)):
        a = y_true[n] - y_pred[n]
        st = a * a
        squ = squ + st
        
    print('Soma dos Quadrados dos Resíduos (SQU):',squ[0])
    print('\n')
    print('Fórmula do R²')
    print('\n')
    print('sqr = stq - squ')
    sqr = stq[0] - squ[0]
    print('R² = sqr/stq')
    sqr = sqr/stq[0]
    print('\n')
    return sqr

def correlacao_determinacao(dtframe,tipo):
    
    if (tipo == 0): #Treino
        resultado = dtframe.corr().previsao_treino.values[1] ** 2 
    else: #teste
        resultado = dtframe.corr().previsao_teste.values[1] ** 2
    
    return resultado

def ajusta_array(array):
    lista = []
    
    for i in range(len(array)):
        lista.append(array[i][0])
        
    #print('ajusta array:',lista)
        
    return lista

def ajusta_lista(array):
    lista = []
    
    for i in range(len(array)):
        lista.append(array[i][0])
        
    #print('ajusta array:',lista)
        
    return lista

In [6]:
df_resultado_medio_g = pd.DataFrame()

index_coluna = 0

parametro = []
tecnica   = []

#média
media_parametro_mape_treino = []
media_parametro_rmse_treino = []
media_parametro_r_treino    = []
    
media_parametro_mape_teste  = []
media_parametro_rmse_teste  = []
media_parametro_r_teste     = []
    
#desvio_padrão
    
std_parametro_mape_treino = []
std_parametro_rmse_treino = []
std_parametro_r_treino    = []
    
std_parametro_mape_teste  = []
std_parametro_rmse_teste  = []
std_parametro_r_teste     = []

for p in ('ph','coliformes', 'dbo','fosforo','od','solido','temperatura','turbidez'):
    
    print('Agora é a vez do parâmetro:', p)
   
    df_resultados_treino = pd.DataFrame()
    df_resultados_teste  = pd.DataFrame()
    df_resultados        = pd.DataFrame()
    
    media_lag_mape_treino = []
    media_lag_rmse_treino = []
    media_lag_r_treino    = []
        
    media_lag_mape_teste = []
    media_lag_rmse_teste = []
    media_lag_r_teste    = []
    
    index_coluna_10 = pega_index_coluna_ugrhi10(p)
    
    #seleciona os dados
    
    train06,train_previsao06, test06 = pegar_dados_coluna_predita_train_test(ugrhi06,0.70,index_coluna) 
    train10,train_previsao10, test10 = pegar_dados_coluna_predita_train_test(ugrhi10,0.70,index_coluna_10)
    
    #print(test10)
    
    #normalização dos dados
    train06,test06,testd06 = normalizacao(train06,test06)
    train10,test10,testd10 = normalizacao(train10,test10)
        
    normalizador_previsao = MinMaxScaler()
    sc = MinMaxScaler()
    normalizador_previsao.fit_transform(train_previsao06)

    for l in range(1, 16):
    
        lags = l

        #Prepara os dados de treinamento -ugrhi06
        #Vai predizer o valor da coluna 4 (valor ph)
        train_X06, train_y06 = prepara_dados(train06, lags,index_coluna)

        #Prepara os dados de teste - ugrhi06
        entradas06 = ugrhi06[len(ugrhi06) - len(test06) - lags:].values
        entradas06 = sc.fit_transform(entradas06)   

        test_X06 = []
        for i in range(lags, lags+len(test06)):
            test_X06.append(entradas06[i-lags:i, 0:8])
        test_X06 = np.array(test_X06)

        #Ajusta a dimensão de test_X06 e train_X06
        n_input = train_X06.shape[1] * train_X06.shape[2]

        test_X06 = test_X06.reshape((test_X06.shape[0],n_input))

        train_X06 = train_X06.reshape((train_X06.shape[0],n_input))

        #Prepara os dados de treinamento -ugrhi06
        #Vai predizer o valor da coluna 4 (valor ph)
        train_X10, train_y10 = prepara_dados(train10, lags,index_coluna_10)

        #Prepara os dados de teste - ugrhi10
        entradas10 = ugrhi10[len(ugrhi10) - len(test10) - lags:].values
        entradas10 = sc.fit_transform(entradas10)   

        test_X10 = []
        for i in range(lags, lags+len(test10)):
            test_X10.append(entradas10[i-lags:i, 0:8])
        test_X10 = np.array(test_X10)

        #Ajusta a dimensão de test_X06 e train_X06
        n_input = train_X10.shape[1] * train_X10.shape[2]

        test_X10 = test_X10.reshape((test_X10.shape[0],n_input))

        train_X10 = train_X10.reshape((train_X10.shape[0],n_input))

        media_simulador_mape_treino = []
        media_simulador_rmse_treino = []
        media_simulador_r_treino    = []

        media_simulador_mape_teste  = []
        media_simulador_rmse_teste  = []
        media_simulador_r_teste     = []
        
        for r in range(0,6):

            #cria o modelo
            model = Sequential()
            model.add(Dense(units = 10, activation = 'relu', input_dim = n_input))
            model.add(Dense(units = 21, activation = 'relu'))
            model.add(Dense(units = 1, activation = 'sigmoid'))
            model.compile(loss = 'mean_absolute_error', optimizer = 'adam',
                          metrics = ['mean_absolute_error'])

            es = EarlyStopping(monitor='val_loss', patience = 3, verbose=0)


            #Treina o modelo
            history = model.fit(train_X06, train_y06, validation_data=(test_X10, test10), batch_size = 32, epochs = 2000, 
                                callbacks=[es], verbose=0)

            #Treina o modelo
            #model.fit(train_X06, train_y06, batch_size = 32, epochs = 100)

            #Dados de teste
            previsoes = model.predict(test_X10)
            #previsoes = previsoes.reshape(-1, 1)
            previsoes = normalizador_previsao.inverse_transform(previsoes)

            #print('Teste - Gráficos com lag', l)
            '''
            #Plotagem do gráfico
            plt.plot(testd10,color='red',label = 'Observado')
            plt.plot(previsoes,color='blue',label = 'Previsoes')
            plt.xlabel('Tempo')
            plt.ylabel('Valor pH')
            plt.legend()
            plt.show()
            '''

            #Dados de treino
            previsoes_treino = model.predict(train_X10)
            previsoes_treino = previsoes_treino.reshape(-1, 1)
            previsoes_treino = normalizador_previsao.inverse_transform(previsoes_treino)

            treino = train_previsao10[lags: len(previsoes_treino) + lags, :]
            observado_test = testd10

            observado_treino = train_previsao10  

            #print('Treinamento - Gráficos com lag', l)
            '''
            #Plotagem do gráfico
            plt.plot(train_previsao10,color='red',label = 'Observado')
            plt.plot(previsoes_treino,color='blue',label = 'Previsoes')
            plt.xlabel('Tempo')
            plt.ylabel('Valor pH')
            plt.legend()
            plt.show()
            '''

            treino         = ajusta_array(treino)
            observado_test = ajusta_array(observado_test)

            #Calculo do erro da previsão MAPE, RMSE e R²

            observado_treino_d = observado_treino
            previsoes_treino_d = previsoes_treino
            previsoes_d        = previsoes

            mape_treino_d        = round(mape(previsoes_treino,treino),4)
            rmse_treino_d        = round(rmse(previsoes_treino,treino),4)
            #r_treino_d           = round(r2(previsoes_treino,treino),4)

            mape_teste_d        = round(mape(previsoes,observado_test),4)
            rmse_teste_d        = round(rmse(previsoes,observado_test),4)
            #r_teste_d           = round(r2(previsoes,observado_test),4) 

            df_corr_determinacao_treino = pd.DataFrame()
            df_corr_determinacao_teste  = pd.DataFrame()

            #Calcula o coeficiente de determinação
            dict = {'previsao_treino': ajusta_lista(previsoes_treino), 'treino': treino} 

            df_treino = pd.DataFrame(dict)
            dframes_treino = [df_corr_determinacao_treino,df_treino]
            df_corr_determinacao_treino = pd.concat(dframes_treino)

            dict = {'previsao_teste': ajusta_lista(previsoes), 'teste': observado_test} 

            df_teste = pd.DataFrame(dict)
            dframes_teste = [df_corr_determinacao_teste,df_teste]
            df_corr_determinacao_teste = pd.concat(dframes_teste)

            r_treino_d = []
            r_teste_d  = []

            r_treino_d.append(round(correlacao_determinacao(df_corr_determinacao_treino,0),4))
            r_teste_d.append(round(correlacao_determinacao(df_corr_determinacao_teste,1),4))
            
            media_simulador_mape_treino.append(np.mean(mape_treino_d))
            media_simulador_rmse_treino.append(np.mean(rmse_treino_d))
            media_simulador_r_treino.append(np.mean(r_treino_d))
            
            media_simulador_mape_teste.append(np.mean(mape_teste_d))
            media_simulador_rmse_teste.append(np.mean(rmse_teste_d))
            media_simulador_r_teste.append(np.mean(r_teste_d))
            
            
    
        media_lag_mape_treino.append(np.mean(media_simulador_mape_treino))
        media_lag_rmse_treino.append(np.mean(media_simulador_rmse_treino))
        media_lag_r_treino.append(np.mean(media_simulador_r_treino))
        
        media_lag_mape_teste.append(np.mean(media_simulador_mape_teste))
        media_lag_rmse_teste.append(np.mean(media_simulador_rmse_teste))
        media_lag_r_teste.append(np.mean(media_simulador_r_teste))
        
                
    #média
    media_parametro_mape_treino.append(np.mean(media_lag_mape_treino))
    media_parametro_rmse_treino.append(np.mean(media_lag_rmse_treino))
    media_parametro_r_treino.append(np.mean(media_lag_r_treino))
    
    media_parametro_mape_teste.append(np.mean(media_lag_mape_teste))
    media_parametro_rmse_teste.append(np.mean(media_lag_rmse_teste))
    media_parametro_r_teste.append(np.mean(media_lag_r_teste))
        
    #desvio_padrão
    
    #std_parametro_mape_treino.append(np.std(media_lag_mape_treino))
    std_parametro_rmse_treino.append(np.std(media_lag_rmse_treino))
    #std_parametro_r_treino.append(np.std(media_lag_r_treino))
    
    #std_parametro_mape_teste.append(np.std(media_lag_mape_teste))
    std_parametro_rmse_teste.append(np.std(media_lag_rmse_teste))
    #std_parametro_r_teste.append(np.std(media_lag_r_teste))
        
    parametro.append(p)
    tecnica.append('MLP (Todas var.)')
    
        
dict = {'parametro': parametro,
         'tecnica': tecnica,
        'mape_treino':  media_parametro_mape_treino,  
        'rmse_treino':  media_parametro_rmse_treino, 'std_rmse_treino':  std_parametro_rmse_treino,
        'r_quad_treino':media_parametro_r_treino,    
        'mape_teste':   media_parametro_mape_teste,   
        'rmse_teste':   media_parametro_rmse_teste,  'std_rmse_teste':   std_parametro_rmse_teste,
        'r_quad_teste': media_parametro_r_teste}
   
df_resultado_final = pd.DataFrame(dict)

dframes = [df_resultado_medio_g,df_resultado_final]
df_resultado_medio_g = pd.concat(dframes)

df_resultado_medio_g.round(2)

Agora é a vez do parâmetro: ph
Agora é a vez do parâmetro: coliformes
Agora é a vez do parâmetro: dbo
Agora é a vez do parâmetro: fosforo
Agora é a vez do parâmetro: od
Agora é a vez do parâmetro: solido
Agora é a vez do parâmetro: temperatura
Agora é a vez do parâmetro: turbidez


,parametro,tecnica,mape_treino,rmse_treino,std_rmse_treino,r_quad_treino,mape_teste,rmse_teste,std_rmse_teste,r_quad_teste
0,ph,MLP (Todas var.),2.03,0.02,0.00,0.03,2.11,0.02,0.0,0.00
1,coliformes,MLP (Todas var.),76.70,3.33,0.01,0.04,72.84,2.95,0.0,0.02
2,dbo,MLP (Todas var.),25.22,0.33,0.00,0.20,26.58,0.37,0.0,0.06
3,fosforo,MLP (Todas var.),490.77,0.66,0.00,0.08,613.18,0.65,0.0,0.01
4,od,MLP (Todas var.),31.24,0.22,0.00,0.03,43.63,0.27,0.0,0.01
5,solido,MLP (Todas var.),62.69,1.51,0.00,0.10,63.36,1.55,0.0,0.01
6,temperatura,MLP (Todas var.),35.26,0.49,0.00,0.01,35.90,0.50,0.0,0.01
7,turbidez,MLP (Todas var.),38.10,0.61,0.00,0.01,38.49,0.64,0.0,0.01


In [7]:
df_resultado_medio_g.round(2).to_csv (r'/home/anderson/Downloads/src_experimentos_dissertação/resultados_log_csv/espaco_temporal_mlp_todas_var.csv', index = True, header=True)